In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from keras import models
from keras import layers
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import EarlyStopping

from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score


Load Dataset 

In [2]:
df=pd.read_csv('editing.csv')

In [3]:
# Display the first few rows of the DataFrame
df.head()

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,required_car_parking_space,lead_time,arrival_year,arrival_month,arrival_date,repeated_guest,...,market_segment_type_0,market_segment_type_1,market_segment_type_2,market_segment_type_3,market_segment_type_4,has_children,co1,co2,co3,co4
0,2,0,1,4,0,118,2017,12,28,0,...,0,1,0,0,0,0,0,0,0,1
1,2,1,0,2,0,17,2018,4,14,0,...,0,1,0,0,0,1,0,1,0,0
2,1,0,1,5,0,349,2018,10,4,0,...,1,0,0,0,0,0,0,0,0,1
3,1,0,2,4,0,69,2018,6,12,0,...,1,0,0,0,0,0,0,1,0,0
4,2,0,0,4,0,11,2018,1,20,0,...,0,1,0,0,0,0,1,0,0,0


Split Data
---


In [4]:
# Drop the 'booking_status' column from the DataFrame to create the features DataFrame X
X = df.drop(columns=['booking_status'])

# Select the 'booking_status' column from the DataFrame to create the target Series y
y = df['booking_status']

# Split the data into training and testing sets, with 25% of the data used for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Print the shapes of the y_train and y_test arrays to verify the split
print(y_train.shape, y_test.shape)


(10519,) (3507,)


In [5]:
# Count the number of canceled bookings in the training set and calculate the percentage
NOcanceled = y_train.sum()
total = y_train.shape[0]
print('Percentage of canceled bookings in the training set:', NOcanceled / total)

# Count the number of canceled bookings in the testing set and calculate the percentage
NOcanceled = y_test.sum()
total = y_test.shape[0]
print('Percentage of canceled bookings in the testing set:', NOcanceled / total)


Percentage of canceled bookings in the training set: 0.2905219127293469
Percentage of canceled bookings in the testing set: 0.2968349016253208


we have outlliers in our data set but we think it is acceptable because there some event the price of hotel ancreased and You must book very early before you arrive Although we well tray modeling in both data with outlier and without it

Remove Outliers
---

In [6]:
# Create copies of the X_train and y_train arrays
X_trainOut = X_train.copy()
y_trainOut = y_train.copy()


In [7]:
# Define a function to remove outliers from a dataset based on a specified column
def remove_outliers(x, y, column):
    # Calculate the first and third quartiles and the interquartile range (IQR)
    Q1 = x[column].quantile(0.25)
    Q3 = x[column].quantile(0.75)
    IQR = Q3 - Q1
    
    # Calculate the lower and upper bounds for outliers using the IQR
    lower_bound = Q1 - 1.5*IQR
    upper_bound = Q3 + 1.5*IQR
    
    # Filter the input data (x and y) to remove outliers based on the specified column
    y = y[(x[column] >= lower_bound) & (x[column] <= upper_bound)]
    x = x[(x[column] >= lower_bound) & (x[column] <= upper_bound)]
    
    # Return the filtered data
    return x, y

# Loop through specified columns and remove outliers from the training data
for col in ['lead_time', 'avg_price_per_room']:
    X_trainOut, y_trainOut = remove_outliers(X_trainOut, y_trainOut, col)

# Print the shape of the original and filtered training data, as well as the shape of the original and filtered labels
print(X_train.shape, X_trainOut.shape, y_train.shape, y_trainOut.shape)


(10519, 35) (9770, 35) (10519,) (9770,)


In [8]:
# Count the number of canceled bookings in the training (without outlier s) set and calculate the percentage
NOcanceled =y_trainOut.sum()
total=y_trainOut.shape[0]
print('Percentage of cancel in train set without outliers',NOcanceled/total)

Percentage of cancel in train set without outliers 0.2804503582395087


Here, we can observe that the percentage of cancellations has decreased after removing the outliers. This was expected to have a negative impact on the model.

Data Scaling
---

In [9]:
# create an instance of the MinMaxScaler class
scaler = MinMaxScaler()

# Scale the training data using fit_transform()
X_trainOut = scaler.fit_transform(X_trainOut)

# Scale the test data using transform()
X_testOut = scaler.transform(X_test)

In [10]:
# create an instance of the MinMaxScaler class
scaler = MinMaxScaler()

# Scale the training data using fit_transform()
X_train = scaler.fit_transform(X_train)

# Scale the test data using transform()
X_test = scaler.transform(X_test)

Now proceed to build different models for both datasets, one with outliers and one without outliers.

Logistic Regression
---

In [11]:
# Create an instance of the LogisticRegression class
log=LogisticRegression()

# Fit the logistic regression model to the training data using the fit() method
log.fit(X_train,y_train)

# Use the predict() method of the logistic regression model to predict the classes of the training data
yd=log.predict(X_train)

# Compute the accuracy of the model on the training data using the accuracy_score() function
print('train accuracy:',accuracy_score(yd,y_train))

# Compute the accuracy of the model on the test data using the accuracy_score() function
yed=log.predict(X_test)
print('test accuracy:',accuracy_score(yed,y_test))

train accuracy: 0.8069208099629243
test accuracy: 0.8006843455945253


c:\Users\OMARS\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [12]:
# Create an instance of the LogisticRegression class
log=LogisticRegression()

# Fit the logistic regression model to the training data using the fit() method
log.fit(X_trainOut,y_trainOut)

# Use the predict() method of the logistic regression model to predict the classes of the training data
yd=log.predict(X_trainOut)

# Compute the accuracy of the model on the training data using the accuracy_score() function
print('train accuracy:',accuracy_score(yd,y_trainOut))

# Compute the accuracy of the model on the test data using the accuracy_score() function
yed=log.predict(X_testOut)
print('test accuracy:',accuracy_score(yed,y_test))

train accuracy: 0.8056294779938588
test accuracy: 0.7966923296264614


c:\Users\OMARS\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Random Forest Classifier
---

In [13]:
# Create an instance of the RandomForestClassifier class with 100 decision trees
rfc = RandomForestClassifier(n_estimators=100)

# Define a dictionary of hyperparameters to tune
param_grid = {'max_depth': range(1, 31)}

# Create a GridSearchCV object with the random forest classifier, hyperparameter grid, and 10-fold cross-validation
grid_search = GridSearchCV(rfc, param_grid=param_grid, cv=10, scoring='accuracy')

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, y_train)

# Print the best set of hyperparameters found by the grid search.
print("Best parameters: ", grid_search.best_params_)

# Evaluate the accuracy of the model on the testing data using the best set of hyperparameters found by the grid search.
print("Accuracy on testing data: ", grid_search.score(X_test, y_test))


Best parameters:  {'max_depth': 16}
Accuracy on testing data:  0.8494439692044482


In [14]:
# Create an instance of the RandomForestClassifier class with 300 decision trees and a maximum depth of 15
rfc = RandomForestClassifier(n_estimators=300, max_depth=16)

# Fit the random forest classifier to the training data using the fit() method
rfc.fit(X_train, y_train)

# Use the predict() method of the random forest classifier to predict the classes of the training data
y_pred = rfc.predict(X_train)

# Compute the accuracy of the model on the training data using the accuracy_score() function
train_acc = accuracy_score(y_train, y_pred)
print('Train accuracy:', train_acc)

# Use the predict() method of the random forest classifier to predict the classes of the test data
y_pred = rfc.predict(X_test)

# Compute the accuracy of the model on the test data using the accuracy_score() function
test_acc = accuracy_score(y_test, y_pred)
print('Test accuracy:', test_acc)


Train accuracy: 0.9163418575910257
Test accuracy: 0.8542914171656687


In [15]:
# Create an instance of the RandomForestClassifier class with 100 decision trees
rfc = RandomForestClassifier(n_estimators=100)

# Define a dictionary of hyperparameters to tune
param_grid = {'max_depth': range(1, 31)}

# Create a GridSearchCV object with the random forest classifier, hyperparameter grid, and 10-fold cross-validation
grid_search = GridSearchCV(rfc, param_grid=param_grid, cv=10, scoring='accuracy')

# Fit the GridSearchCV object to the training data
grid_search.fit(X_trainOut, y_trainOut)

# Print the best hyperparameters found by GridSearchCV
print("Best parameters: ", grid_search.best_params_)

# Print the accuracy of the model on the test data using the best hyperparameters found by GridSearchCV
print("Accuracy on testing data: ", grid_search.score(X_testOut, y_test))


Best parameters:  {'max_depth': 15}
Accuracy on testing data:  0.846307385229541


In [16]:
# Create an instance of the RandomForestClassifier class with 300 decision trees and a maximum depth of 15
rfc = RandomForestClassifier(n_estimators=300, max_depth=16)

# Fit the random forest classifier to the training data
rfc.fit(X_trainOut, y_trainOut)

# Use the fitted model to make predictions on the training data and print the accuracy
y_pred = rfc.predict(X_trainOut)
print('train accuracy:', accuracy_score(y_trainOut, y_pred))

# Use the fitted model to make predictions on the test data and print the accuracy
y_pred = rfc.predict(X_testOut)
print('test accuracy:', accuracy_score(y_test, y_pred))


train accuracy: 0.9165813715455476
test accuracy: 0.8488736812090105


KNeighbors Classifier
----

In [17]:
# Create an instance of the KNeighborsClassifier class
knn = KNeighborsClassifier()

# Set the range of hyperparameter values to test for the number of neighbors, and use 10-fold cross validation
param_grid = {'n_neighbors': range(1, 11)}
grid_search = GridSearchCV(knn, param_grid, cv=10, scoring='accuracy')

# Fit the grid search object to the training data
grid_search.fit(X_train, y_train)

# Print the best value of n_neighbors found by the grid search
print("Best value of n_neighbors:", grid_search.best_params_)

# Calculate the accuracy of the KNN classifier using the best value of n_neighbors found by the grid search
accuracy = grid_search.score(X_test, y_test)
print("Accuracy of KNN classifier with best n_neighbors:", accuracy)


Best value of n_neighbors: {'n_neighbors': 9}
Accuracy of KNN classifier with best n_neighbors: 0.799258625605931


In [18]:
# Create an instance of the KNeighborsClassifier class with n_neighbors=9
knn = KNeighborsClassifier(n_neighbors=9)

# Fit the KNN model to the training data
knn.fit(X_train, y_train)

# Make predictions on the training data and calculate the accuracy
y_pred = knn.predict(X_train)
print('train accuracy:' ,accuracy_score(y_train, y_pred))

# Make predictions on the testing data and calculate the accuracy
y_pred = knn.predict(X_test)
print('test accuracy:' ,accuracy_score(y_test, y_pred))


train accuracy: 0.8342998383876794
test accuracy: 0.799258625605931


In [19]:
# Create an instance of the KNeighborsClassifier class
knn = KNeighborsClassifier()

# Define a dictionary of parameter values to be searched over
param_grid = {'n_neighbors': range(1, 11)}

# Create an instance of GridSearchCV with 10-fold cross-validation and 'accuracy' as the scoring metric
grid_search = GridSearchCV(knn, param_grid, cv=10, scoring='accuracy')

# Fit the GridSearchCV object to the training data
grid_search.fit(X_trainOut, y_trainOut)

# Print the best value of n_neighbors found during the grid search
print("Best value of n_neighbors:", grid_search.best_params_)

# Calculate the accuracy of the best KNN classifier on the testing data
accuracy = grid_search.score(X_testOut, y_test)
print("Accuracy of KNN classifier with best n_neighbors:", accuracy)


Best value of n_neighbors: {'n_neighbors': 9}
Accuracy of KNN classifier with best n_neighbors: 0.7909894496720844


In [20]:
# Import the KNeighborsClassifier class from scikit-learn library
from sklearn.neighbors import KNeighborsClassifier

# Create an instance of the KNeighborsClassifier class with n_neighbors=9
knn = KNeighborsClassifier(n_neighbors=9)

# Fit the KNN classifier to the training data
knn.fit(X_trainOut, y_trainOut)

# Use the KNN classifier to make predictions on the training data and calculate the accuracy
y_pred = knn.predict(X_trainOut)
print('train accuracy:' ,accuracy_score(y_trainOut, y_pred))

# Use the KNN classifier to make predictions on the testing data and calculate the accuracy
y_pred = knn.predict(X_testOut)
print('test accuracy:' ,accuracy_score(y_test, y_pred))


train accuracy: 0.8425793244626407
test accuracy: 0.7909894496720844


Support Vector Machine (poly)
---

In [21]:
# define the parameters to search over
parameters = {'kernel':['poly'], 'degree':range(2,11)}

# initialize the SVM classifier
svmc = SVC()

# perform a grid search with 10-fold cross-validation and accuracy scoring
clf = GridSearchCV(svmc, parameters, cv=10, scoring='accuracy')

# fit the model to the training data
clf.fit(X_train, y_train)

# print the best parameters found by the grid search
print("Best parameters:", clf.best_params_)

# predict the labels of the training data and compute the accuracy
y_pred = clf.predict(X_train)
print('train accuracy:', accuracy_score(y_train, y_pred))

# predict the labels of the testing data and compute the accuracy
y_pred = clf.predict(X_test)
print('test accuracy:', accuracy_score(y_test, y_pred))


Best parameters: {'degree': 8, 'kernel': 'poly'}
train accuracy: 0.8505561365148778
test accuracy: 0.8177929854576561


In [24]:
# define the parameters to search over
parameters = {'kernel':['poly'], 'degree':range(2,11)}

# initialize the SVM classifier
svmc = SVC()

# perform a grid search with 10-fold cross-validation and accuracy scoring
clf = GridSearchCV(svmc, parameters, cv=10, scoring='accuracy')

# fit the model to the training data
clf.fit(X_trainOut, y_trainOut)

# print the best parameters found by the grid search
print("Best parameters:", clf.best_params_)

# predict the labels of the training data and compute the accuracy
y_pred = clf.predict(X_trainOut)
print('train accuracy:', accuracy_score(y_trainOut, y_pred))

# predict the labels of the testing data and compute the accuracy
y_pred = clf.predict(X_testOut)
print('test accuracy:', accuracy_score(y_test, y_pred))


Best parameters: {'degree': 9, 'kernel': 'poly'}
train accuracy: 0.8623336745138178
test accuracy: 0.8177929854576561


Gradient Boosting Classifier
---

In [25]:
# Initialize Gradient Boosting Classifier with 100 estimators and a random state of 40
gb_clf = GradientBoostingClassifier(n_estimators=100, random_state=40)

# Define the grid of hyperparameters to search over
param_grid = {'learning_rate': [0.1], 'max_depth': range(2, 10), 'max_features': range(4, 12)}

# Perform grid search using 5-fold cross validation and accuracy as the scoring metric
grid_search = GridSearchCV(gb_clf, param_grid, cv=5, scoring='accuracy', verbose=0)

# Fit the grid search to the training data
grid_search.fit(X_train, y_train)

# Print the best hyperparameters found by the grid search
print("Best parameters:", grid_search.best_params_)

# Calculate and print the accuracy of the model on the testing data using the best hyperparameters found by the grid search
accuracy = grid_search.score(X_test, y_test)
print("Accuracy of Gradient Boosting Classifier with best parameters:", accuracy)


Best parameters: {'learning_rate': 0.1, 'max_depth': 6, 'max_features': 9}
Accuracy of Gradient Boosting Classifier with best parameters: 0.8545765611633875


In [26]:
# Create a Gradient Boosting Classifier with specified hyperparameters
gb_clf = GradientBoostingClassifier(n_estimators=300, learning_rate=0.1, max_features=9, max_depth=6, random_state=40)

# Fit the classifier to the training data
gb_clf.fit(X_train, y_train)

# Make predictions on the training data and calculate the accuracy
y_pred = gb_clf.predict(X_train)
accuracy = accuracy_score(y_train, y_pred)
print('train accuracy:' ,accuracy) 

# Make predictions on the test data and calculate the accuracy
y_pred = gb_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('test accuracy:' ,accuracy)


train accuracy: 0.9360205342713186
test accuracy: 0.8545765611633875


In [27]:
# Create a Gradient Boosting Classifier object with 100 estimators
gb_clf = GradientBoostingClassifier(n_estimators=100, random_state=40)

# Define the parameter grid to search over
param_grid = {'learning_rate': [0.1],'max_depth':range(2, 10), 'max_features':range(4, 12)}

# Perform a grid search using 5-fold cross-validation
grid_search = GridSearchCV(gb_clf, param_grid, cv=5, scoring='accuracy', verbose=0)

# Fit the grid search object on the oversampled training data
grid_search.fit(X_trainOut, y_trainOut)

# Print the best parameters found by the grid search
print("Best parameters:", grid_search.best_params_)

# Calculate the accuracy of the Gradient Boosting Classifier on the oversampled testing data using the best parameters
accuracy = grid_search.score(X_testOut, y_test)

# Print the accuracy score
print("Accuracy of Gradient Boosting Classifier with best parameters:", accuracy)


Best parameters: {'learning_rate': 0.1, 'max_depth': 7, 'max_features': 11}
Accuracy of Gradient Boosting Classifier with best parameters: 0.8485885372112917


In [28]:
# Create an instance of the GradientBoostingClassifier model with the specified hyperparameters
gb_clf = GradientBoostingClassifier(n_estimators=300, learning_rate=0.1, max_features=11, max_depth=6, random_state=40)

# Fit the model to the training data
gb_clf.fit(X_trainOut, y_trainOut)

# Use the trained model to predict the target variable of the training data
y_pred = gb_clf.predict(X_trainOut)

# Calculate the accuracy of the model on the training data
accuracy = accuracy_score(y_trainOut, y_pred)

# Print the accuracy of the model on the training data
print('train accuracy:', accuracy) 

# Use the trained model to predict the target variable of the testing data
y_pred = gb_clf.predict(X_testOut)

# Calculate the accuracy of the model on the testing data
accuracy = accuracy_score(y_test, y_pred)

# Print the accuracy of the model on the testing data
print('test accuracy:', accuracy)


train accuracy: 0.9471852610030707
test accuracy: 0.8448816652409467


Based on the previous models, it was found that the dataset with outliers produced better results than the dataset without outliers. As we anticipated, going forward, we will be using only the dataset with outliers for the purpose of project time scope.

Neural Network
--------

In [29]:
# Split the training data into a partial training set and a validation set
partial_X_train , X_val, partial_y_train , y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [30]:
# Define the neural network model
model = models.Sequential([
  layers.Dense(64, activation='relu', input_dim=partial_X_train.shape[1]),
  layers.Dense(32, activation='relu'),
  layers.Dense(16, activation='relu'),
  layers.Dense(1, activation='sigmoid')
])

# Compile the model with the rmsprop optimizer, binary crossentropy loss function, and accuracy metric
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

# Set up early stopping based on validation accuracy with a patience of 30 epochs
es = EarlyStopping(monitor='val_accuracy', patience=30, mode='max')


In [31]:
# Train the model using the training data and validation data
history = model.fit(partial_X_train, partial_y_train, epochs=1000, batch_size=512, verbose=0,callbacks=[es], validation_data=(X_val, y_val))

In [33]:
# Make predictions using the trained model on the test data
y_pred = model.predict(X_test)

# Convert the probabilities to binary predictions using a threshold of 0.5
y_pred_binary = (y_pred > 0.5).astype(int)

# Calculate the accuracy on the test data
accuracy = (y_pred_binary[:,0] == y_test).mean()

# Print the test accuracy
print("Test Accuracy:", accuracy)


110/110 [==============================] - 0s 813us/step
Test Accuracy: 0.812375249500998


It has been determined that the top three models are the GradientBoostingClassifier, RandomForestClassifier, and Support Vector Machine. To create the final model, we will be using a Voting Classifier that combines these three models.

Final model :Voting Classifier
---

In [40]:
clf1 = GradientBoostingClassifier(n_estimators=300, learning_rate=0.1, max_features=9, max_depth=6, random_state=40)
clf2 = RandomForestClassifier(n_estimators=300,max_depth=14)
clf3 = SVC(kernel='poly',degree=8, probability=True)

voting_clf = VotingClassifier(estimators=[('gb', clf1), ('rf', clf2),('sv',clf3)], voting='soft')

cv_scores = cross_val_score(voting_clf, X_train, y_train, cv=10)
print("Cross-validation scores:", cv_scores)
print("Mean cross-validation score:", cv_scores.mean())

voting_clf.fit(X_train, y_train)
y_pred = voting_clf.predict(X_train)

accuracy = accuracy_score(y_train, y_pred)
print("train accuracy:", accuracy)

y_pred = voting_clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Test accuracy:", accuracy)

Cross-validation scores: [0.86977186 0.85646388 0.85741445 0.84220532 0.84885932 0.84505703
 0.84220532 0.85171103 0.84695817 0.84205519]
Mean cross-validation score: 0.8502701573370283
train accuracy: 0.902272079094971
Test accuracy: 0.8554319931565441
